In [3]:
!pip install sklearn spacy neo4j neomodel nltk
!pip install pandas Flask flask_cors
!python -m spacy download 'en_core_web_lg'

In [4]:
from DataSetupPackage import DataSetUpPackage
from DataAccessLayer import DataAccessLayer
import json
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [5]:
dbcon = DataAccessLayer(username='neo4j',password='neo123456').CreateDBConnection
dbcon.ClearCurrentDB  # This will clean up the neo4jDB
DataSetUpPackage().ReadCSVAndPopulateDB.PopulateNeo4j(dbcon.Session, verbose=True)

### Visualize created knowledge graph at: http://localhost:7474/browser/

* Username: neo4j

* Password: neo123456

In [2]:
dbcon = DataAccessLayer(username='neo4j',password='neo123456').CreateDBConnection
result2 = dbcon.GetAllNodeListOfType('Disease')
for resulti in result2:
    print("Disease Name: " + resulti.name)
    print(resulti.environment)
    for r in resulti.symptoms:
        print(r.name + " # " +str(r.id) +" === " + str(resulti.symptoms.relationship(r).weight) + str(resulti.symptoms.relationship(r).penalty))
    print("----------------------------")

Disease Name: Columnaris
Freshwater
COTTONGROWTH # 23 === 1.0-0.2
GREYMOUTH # 22 === 1.0-0.2
LOSSAPPETITE # 233 === 1.0-0.2
RAPIDBREATH # 21 === 1.0-0.2
BROWNGILLS # 20 === 1.0-0.2
MUCUSHEAD # 19 === 1.0-0.2
GREYGILL # 18 === 1.0-0.2
SKINPEEL # 17 === 1.0-0.2
SKINULCER # 14 === 1.0-0.2
FRAYFIN # 197 === 1.0-0.2
----------------------------
Disease Name: Dropsy
ALL
RAPIDGILL # 34 === 1.0-0.2
LOSSAPPETITE # 233 === 1.0-0.2
LETHARGIC # 33 === 1.0-0.2
EYEPROD # 32 === 1.0-0.2
SCALEOUT # 31 === 1.0-0.2
BLOAT # 30 === 1.0-0.2
----------------------------
Disease Name: Fish Tuberculosis
ALL
HIDING # 42 === 1.0-0.2
LOSSAPPETITE # 233 === 1.0-0.2
YELLOWNODULE # 41 === 1.0-0.2
FINSCALELOSS # 40 === 1.0-0.2
FRAYFIN # 197 === 1.0-0.2
SKINULCER # 14 === 1.0-0.2
PALESKIN # 39 === 1.0-0.2
HOLLOWBELLY # 38 === 1.0-0.2
LETHARGIC # 33 === 1.0-0.2
----------------------------
Disease Name: Viral Hemorrhagic Septicemia 
ALL
PALEGILL # 50 === 1.0-0.2
ERSWIM # 49 === 1.0-0.2
GASPING # 48 === 1.0-0.2
RAPIDBR

In [3]:
newCase = dbcon.CreateOrGetCaseNode('case1')
print(newCase.name + " # " + str(newCase.id))

case1 # 215


In [5]:
symptom1 = dbcon.GetOneSymptomNode("WORMBODY")
print(symptom1.id)

86


In [5]:
dbcon = DataAccessLayer(username='neo4j',password='ai-user').CreateDBConnection
newCase = dbcon.CreateOrGetCaseNode('case1',None,True)
symptom1 = dbcon.GetOneSymptomNode("WORMBODY")
newCase = dbcon.UpdateSymptomToCase(newCase, symptom1, 0.5)
print(len(newCase.suspected_symptoms))
for r in newCase.suspected_symptoms:
    print(r.name + " # " +str(r.id) +" === " + str(newCase.suspected_symptoms.relationship(r).suspectedLevel))

1
WORMBODY # 86 === 0.5


In [6]:
for d in symptom1.diseases:
    dbcon.UpdateDiseaseToCase(newCase, d, 1/len(d.symptoms))
    print(d.name)

Anchor Worms


In [7]:
symptom2 = dbcon.GetOneSymptomNode("GASPING")
newCase = dbcon.UpdateSymptomToCase(newCase, symptom2)
for d in symptom2.diseases:
    confidence = 1/len(d.symptoms)
    rel = newCase.suspected_diseases.relationship(d)
    if rel:
        confidence += rel.confidence
    dbcon.UpdateDiseaseToCase(newCase, d, confidence)
    print(d.name + " # " + str(confidence))

Gill Mite # 0.16666666666666666
CO2 Poisoning # 0.3333333333333333
Ammonia Poisoning # 0.5
Swim Bladder Disease # 0.125
Viral Hemorrhagic Septicemia  # 0.1111111111111111


In [9]:
symptom2

<Symptom: {'name': 'GASPING', 'description': 'Gasping at the surface', 'type': 'Behaviourial', 'tm_token': ['gasp', 'surface'], 'category1': 'Breathing', 'category2': 'Movement', 'category3': 'nan', 'imageurl': 'nan', 'question': 'Is your fish gasping for air at the water surface?', 'id': 48}>